In [ ]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

In [ ]:
!pip install streamlit langchain-community langchain-openai python-dotenv ngrok openai pandas

In [ ]:
!pip install psycopg2

In [ ]:
!pip install pyngrok

In [ ]:
import getpass
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LC_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('GPT_KEY')
db_name = userdata.get('db_name')
db_user = userdata.get('db_user')
db_password = userdata.get('db_password')
db_port = userdata.get('db_port')
db_host = userdata.get('db_host')

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install faiss-cpu

In [ ]:
import os
os.environ["OPENAI_API_KEY"] =""
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import getpass
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""
db_user = ""
db_password = ""
db_host = ""
db_port = ""
db_name = ""


from langchain_community.utilities.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.memory import ChatMessageHistory
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import streamlit as st
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
import streamlit as st
from openai import OpenAI
#from langchain_utils import invoke_chain


llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
st.set_page_config(page_title="Chat with Opioid Data", page_icon=":pill:", layout="wide")


table_info = """
CREATE TABLE county_data (
	"Region" INTEGER,
	"County" VARCHAR(100) NOT NULL,
	"Number of Opioid Fatalities" INTEGER,
	"Opioid Fatality Rate per 100K Capita" DOUBLE PRECISION,
	"Census" INTEGER,
	"State Population Percentage" DOUBLE PRECISION,
	"State Pop Rank" INTEGER,
	"State Opioid Fatality Rank" INTEGER,
	"Social Vulnerability Index (SVI) Score" DOUBLE PRECISION,
	"Number of Opioid Overdose Emerg Dpt Cases" INTEGER,
	"White Rate" DOUBLE PRECISION,
	"White Population" INTEGER,
	"Black Rate" DOUBLE PRECISION,
	"Black Population" INTEGER,
	"Latino Rate" DOUBLE PRECISION,
	"Latino Population" INTEGER,
	"Asian Rate" DOUBLE PRECISION,
	"Asian Population" INTEGER,
	"Native Hawaiian Rate" DOUBLE PRECISION,
	"Native Hawaiian Population" INTEGER,
	"Indian/Alaskan Native Rate" DOUBLE PRECISION,
	"Indian/Alaskan Native Population" INTEGER,
	"Aged <15 Rate" DOUBLE PRECISION,
	"Aged <15 Population" INTEGER,
	"Aged 15-64 Rate" DOUBLE PRECISION,
	"Aged 15-64 Population" INTEGER,
	"Aged 65+ Rate" DOUBLE PRECISION,
	"Aged 65+ Population" INTEGER,
	"Completed High School Rate" DOUBLE PRECISION,
	"Completed High School Population" INTEGER,
	"Completed College Rate" DOUBLE PRECISION,
	"Completed College Population" INTEGER,
	"Aged 3-4 School Enrolled Rate" DOUBLE PRECISION,
	"Aged 3-4 School Enrolled Population" INTEGER,
	"Median Household Income" INTEGER,
	"Labour Force Participation Rate" DOUBLE PRECISION,
	"Labour Force Population" INTEGER,
	"Poverty Rate" DOUBLE PRECISION,
	"Population in Poverty" INTEGER,
	"Recovery Ecosystem Index score" INTEGER,
	"Substance Use Disorder (SUD) Treatment score" INTEGER,
	"Continuum of SUD Support score" INTEGER,
	"Infrastructure and Social Factors score" INTEGER,
	"One or More Vehicles Rate" DOUBLE PRECISION,
	"Population With One or More Vehicles" INTEGER,
	"Broadband Access Rate" DOUBLE PRECISION,
	"Population with Broadband Access" INTEGER,
	"Social Associations Rate" DOUBLE PRECISION,
	"Population with Social Associations" INTEGER,
	"Severe Housing Cost Burden Rate" DOUBLE PRECISION,
	"Population with Housing Cost Burden" INTEGER,
	"Substance Use Treatment Facilities" INTEGER,
	"Buprenorphine Providers" INTEGER,
	"Average Distance to Nearest MAT Provider (miles)" INTEGER,
	"Mental Health Providers" INTEGER,
	"Recovery Residences" INTEGER,
	"NA or SMART Meetings per 100k" INTEGER,
	"Average Distance to Nearest SSP (miles)" INTEGER,
	"Drug-Free Communities Coalition" BOOLEAN,
	"Drug Courts" BOOLEAN,
	"Frequent Mental Distress Rate" DOUBLE PRECISION,
	"Drug Overdose Deaths" INTEGER,
	"Drug Overdose Mortality Rate" DOUBLE PRECISION,
	"Drug Overdose Mortality Rate (Native)" DOUBLE PRECISION,
	"Drug Overdose Mortality Rate (Asian)" DOUBLE PRECISION,
	"Drug Overdose Mortality Rate (Black)" DOUBLE PRECISION,
	"Drug Overdose Mortality Rate (Hispanic)" DOUBLE PRECISION,
	"Drug Overdose Mortality Rate (White)" DOUBLE PRECISION,
	"Uninsured Adults Rate" DOUBLE PRECISION,
	"Disconnected Youth" DOUBLE PRECISION,
	"Homicide Rate (IL)" DOUBLE PRECISION,
	"Homicide Rate (Black)" DOUBLE PRECISION,
	"Homicide Rate (Hispanic)" DOUBLE PRECISION,
	"Homicide Rate (White)" DOUBLE PRECISION,
	"Firearm Fatalities Rate (Black)" DOUBLE PRECISION,
	"Firearm Fatalities Rate (Hispanic)" DOUBLE PRECISION,
	"Juvenile Arrest Rate" DOUBLE PRECISION,
	"Rural Rate" DOUBLE PRECISION,
	CONSTRAINT county_data_pkey PRIMARY KEY ("County")
)
"""
examples = [
    {
        "input": "How many people live in each county?",
        "StructurePrompt": "SELECT [column1], [column2] FROM [table];",
        "ContentPrompt": "SELECT \"County\",\"Census\" FROM county_data;"
    },
    {
        "input": "Show the opioid fatality rates for all counties.",
        "StructurePrompt": "SELECT [column1], [column2] FROM [table];",
        "ContentPrompt": "SELECT \"County\", \"Opioid Fatality Rate per 100K Capita\" FROM county_data;"
    },
    {
        "input": "What is the number of emergency department cases for opioid overdose in Broward County?",
        "StructurePrompt": "SELECT [column] FROM [table] WHERE [condition_column] = '[value]';",
        "ContentPrompt": "SELECT \"Number of Opioid Overdose Emerg Dpt Cases.\" FROM county_data WHERE \"County\" = 'Broward County';"
    },
    {
        "input": "List the counties where there have been more than 1,000 opioid fatalities.",
        "StructurePrompt": "SELECT [column] FROM [table] WHERE [column] > [value];",
        "ContentPrompt": "SELECT \"County\" FROM county_data WHERE \"Number of Opioid Fatalities\" > 1000;"
    },
    {
        "input": "Identify counties with broadband access rates below 50%.",
        "StructurePrompt": "SELECT [column] FROM [table] WHERE [column] < [value];",
        "ContentPrompt": "SELECT \"County\" FROM county_data WHERE \"Broadband Access Rate\" < 0.5;"
    },
    {
        "input": "Which counties are ranked in the top 10 for opioid fatalities?",
        "StructurePrompt": "SELECT [column] FROM [table] ORDER BY [column] LIMIT [value];",
        "ContentPrompt": "SELECT \"County\" FROM county_data ORDER BY \"Number of Opioid Fatalities\" LIMIT 10;"
    },
    {
        "input": "Find counties and their numbers with high opioid fatalities and low college completion rates.",
        "StructurePrompt": "SELECT [column1],[column2],[column3] FROM [table] WHERE [column1] > [value1] AND [column2] < [value2];",
        "ContentPrompt": "SELECT \"County\",\"Number of Opioid Fatalities\",\"Completed College Rate\" FROM county_data WHERE \"Number of Opioid Fatalities\" > 50 AND \"Completed College Rate\" < 0.3;"
    },
    {
        "input": "Compare the social vulnerability index with the opioid overdose death rate in each county.",
        "StructurePrompt": "SELECT [column1], [column2], [column3] FROM [table];",
        "ContentPrompt": "SELECT \"County\", \"Social Vulnerability Index (SVI) Score\", \"Drug Overdose Mortality Rate\" FROM county_data;"
    },
    {
       "input": "List counties where the poverty rate exceeds 20% and the drug overdose mortality rate is above the median.",
       "StructurePrompt": "WITH [temp_table] AS (SELECT [aggregation_function] WITHIN GROUP (ORDER BY [column1]) AS [alias1] FROM [table]) SELECT [table_alias].[column2], [table_alias].[column3], [table_alias].[column1] FROM [table] [table_alias], [temp_table] [temp_table_alias] WHERE [table_alias].[column3] > [value1] AND [table_alias].[column1] > [temp_table_alias].[alias1];",
       "ContentPrompt": "WITH MedianMortality AS (SELECT percentile_cont(0.5) WITHIN GROUP (ORDER BY \"Drug Overdose Mortality Rate\") AS MedianRate FROM county_data) SELECT c.\"County\", c.\"Poverty Rate\", c.\"Drug Overdose Mortality Rate\" FROM county_data c, MedianMortality m WHERE c.\"Poverty Rate\" > 0.2 AND c.\"Drug Overdose Mortality Rate\" > m.MedianRate;"
    },
    {
       "input": "Assess the correlation between opioid fatalities and labor force participation in counties with high social vulnerability.",
       "StructurePrompt": "SELECT [aggregation_function]([table_alias].[column1], [table_alias].[column2]) AS [alias] FROM [table] [table_alias] WHERE [table_alias].[column3] > [value];",
       "ContentPrompt": "SELECT corr(c.\"Number of Opioid Fatalities\", c.\"Labour Force Participation Rate\") AS CorrelationCoefficient FROM county_data c WHERE c.\"Social Vulnerability Index (SVI) Score\" > 0.8;"
    },
    {
        "input": "Explore the relationship between the number of drug courts and opioid fatality rates in rural counties.",
        "StructurePrompt": "SELECT [column1],[column2],[column3] FROM [table] WHERE [column4] > [value];",
        "ContentPrompt": "SELECT \"County\",\"Drug Courts\", \"Opioid Fatality Rate per 100K Capita\" FROM county_data WHERE \"Rural Rate\" > 0.7;"
    },
    {
       "input": "Evaluate the impact of drug-free community coalitions on opioid fatalities in the ten most populous counties.",
       "StructurePrompt": "WITH [temp_table] AS (SELECT [column1] FROM [table] ORDER BY [column2] DESC LIMIT [value]) SELECT [table_alias].[column1], [table_alias].[column3], [table_alias].[column4] FROM [table] [table_alias] JOIN [temp_table] [temp_table_alias] ON [table_alias].[column1] = [temp_table_alias].[column1] GROUP BY [table_alias].[column1], [table_alias].[column3];",
       "ContentPrompt": "WITH TopCounties AS (SELECT \"County\" FROM county_data ORDER BY \"Census\" DESC LIMIT 10) SELECT c.\"County\", c.\"Drug-Free Communities Coalition\", c.\"Number of Opioid Fatalities\" FROM county_data c JOIN TopCounties tc ON c.\"County\" = tc.\"County\" GROUP BY c.\"County\", c.\"Drug-Free Communities Coalition\";"
    },
    {
      "input": "Determine the effectiveness of buprenorphine providers in reducing the number of opioid fatalities.",
      "StructurePrompt": "SELECT [aggregation_function]([column1], [column2]) AS [alias] FROM [table];",
      "ContentPrompt": "SELECT corr(\"Buprenorphine Providers\", \"Number of Opioid Fatalities\") AS CorrelationCoefficient FROM county_data;"
    },
    {
       "input": "Analyze the influence of median household income on the rates of uninsured adults in counties with high opioid fatalities.",
       "StructurePrompt": "SELECT [column1], [column2] FROM [table] WHERE [column3] > [value] ORDER BY [column1];",
       "ContentPrompt": "SELECT \"Median Household Income\", \"Uninsured Adults Rate\" FROM county_data WHERE \"Number of Opioid Fatalities\" > 100 ORDER BY \"Median Household Income\";"
    },
    {
       "input": "Correlate the availability of mental health providers with the frequency of mental distress and opioid overdose deaths.",
       "StructurePrompt": "SELECT [aggregation_function1]([column1], [column2]) AS [alias1], [aggregation_function2]([column1], [column3]) AS [alias2] FROM [table];",
       "ContentPrompt": "SELECT corr(\"Mental Health Providers\", \"Frequent Mental Distress Rate\") AS CorrelationDistress, corr(\"Mental Health Providers\", \"Drug Overdose Deaths\") AS CorrelationOverdoseDeaths FROM county_data;"
    },
    {
       "input": "Examine the relationship between juvenile arrest rates and drug overdose mortality in urban versus rural counties.",
       "StructurePrompt": "SELECT [column1], [column2], [column3] FROM [table] WHERE [column1] >= [value] OR [column1] < [value] ORDER BY [column1];",
       "ContentPrompt": "SELECT \"Rural Rate\", \"Juvenile Arrest Rate\", \"Drug Overdose Mortality Rate\" FROM county_data WHERE \"Rural Rate\" >= 0.7 OR \"Rural Rate\" < 0.7 ORDER BY \"Rural Rate\";"
    },
    {
       "input": "Evaluate the effectiveness of mental health services by comparing opioid fatalities in counties with above-average mental health providers to the state average.",
       "StructurePrompt": "WITH [temp_table1] AS (SELECT AVG([column1]) AS [alias1] FROM [table]), [temp_table2] AS (SELECT AVG([column2]) AS [alias2] FROM [table]) SELECT [table_alias].[column3], [table_alias].[column2], [temp_table1_alias].[alias1], [temp_table2_alias].[alias2] FROM [table] [table_alias], [temp_table1] [temp_table1_alias], [temp_table2] [temp_table2_alias] WHERE [table_alias].[column1] > [temp_table1_alias].[alias1] ORDER BY [table_alias].[column2] DESC;",
       "ContentPrompt": "WITH AverageProviders AS (SELECT AVG(\"Mental Health Providers\") AS AvgProviders FROM county_data), StateAverageFatalities AS (SELECT AVG(\"Number of Opioid Fatalities\") AS AvgFatalities FROM county_data) SELECT c.\"County\", c.\"Number of Opioid Fatalities\", m.AvgProviders, s.AvgFatalities FROM county_data c, AverageProviders m, StateAverageFatalities s WHERE c.\"Mental Health Providers\" > m.AvgProviders ORDER BY c.\"Number of Opioid Fatalities\" DESC;"
    },
    {
       "input": "Determine the correlation between opioid fatalities and socioeconomic factors like unemployment and high school non-completion rates.",
       "StructurePrompt": "SELECT [aggregation_function1]([column1], [column2]) AS [alias1], [aggregation_function2]([column1], [column3]) AS [alias2] FROM [table];",
       "ContentPrompt": "SELECT corr(\"Number of Opioid Fatalities\", \"Labour Force Participation Rate\") AS CorrelationWithUnemployment, corr(\"Number of Opioid Fatalities\", \"Completed High School Rate\") AS CorrelationWithHighSchoolCompletion FROM county_data;"
    },
    {
       "input": "Analyze the relationship between the availability of recovery residences and opioid fatality rates across urban and rural settings.",
       "StructurePrompt": "SELECT [column1], [column2], [column3], [aggregation_function]([column2], [column3]) OVER (PARTITION BY CASE WHEN [column1] >= [value] THEN 'Rural' ELSE 'Urban' END) AS [alias] FROM [table] ORDER BY [column1];",
       "ContentPrompt": "SELECT \"Rural Rate\", \"Recovery Residences\", \"Number of Opioid Fatalities\", corr(\"Recovery Residences\", \"Number of Opioid Fatalities\") OVER (PARTITION BY CASE WHEN \"Rural Rate\" >= 0.5 THEN 'Rural' ELSE 'Urban' END) AS CorrelationBySetting FROM county_data ORDER BY \"Rural Rate\";"
    },
    {
       "input": "List counties based on high and low vehicle ownership rates.",
       "StructurePrompt": "SELECT (SELECT [array_function]([column1]) FROM [table] WHERE [column2] > [value1]) AS [alias1], (SELECT [array_function]([column1]) FROM [table] WHERE [column2] < [value2]) AS [alias2];",
       "ContentPrompt": "SELECT (SELECT array_agg(\"County\") FROM county_data WHERE \"One or More Vehicles Rate\" > 0.9) AS \"High Vehicle Ownership Counties\", (SELECT array_agg(\"County\") FROM county_data WHERE \"One or More Vehicles Rate\" < 0.5) AS \"Low Vehicle Ownership Counties\";"
    },
    {
       "input": "Analyze the correlation between educational achievement and opioid fatalities in counties with a Black population.",
       "StructurePrompt": "SELECT [column1], [column2], [column3], [column4], [correlation_function]([column2], [column4]) OVER () AS [alias1], [correlation_function]([column3], [column4]) OVER () AS [alias2] FROM [table] WHERE [column5] > [value] ORDER BY [column2], [column3];",
       "ContentPrompt": "SELECT \"County\", \"Completed High School Rate\", \"Completed College Rate\", \"Number of Opioid Fatalities\", corr(\"Completed High School Rate\", \"Number of Opioid Fatalities\") OVER () AS CorrelationWithHighSchoolCompletion, corr(\"Completed College Rate\", \"Number of Opioid Fatalities\") OVER () AS CorrelationWithCollegeCompletion FROM county_data WHERE \"Black Population\" > 0 ORDER BY \"Completed High School Rate\", \"Completed College Rate\";"
    },
    {
       "input": "Compare opioid fatality rates between counties with a high number of substance use treatment facilities and those with a low number.",
       "StructurePrompt": "WITH [temp_table1] AS (SELECT AVG([column1]) AS [alias1] FROM [table] WHERE [column2] > [value1]), [temp_table2] AS (SELECT AVG([column1]) AS [alias2] FROM [table] WHERE [column2] < [value2]) SELECT [temp_table1_alias].[alias1], [temp_table2_alias].[alias2] FROM [temp_table1] [temp_table1_alias], [temp_table2] [temp_table2_alias];",
       "ContentPrompt": "WITH HighTreatmentFacilities AS (SELECT AVG(\"Opioid Fatality Rate per 100K Capita\") AS AvgHighTreatmentMortalityRate FROM county_data WHERE \"Substance Use Treatment Facilities\" > 20), LowTreatmentFacilities AS (SELECT AVG(\"Opioid Fatality Rate per 100K Capita\") AS AvgLowTreatmentMortalityRate FROM county_data WHERE \"Substance Use Treatment Facilities\" < 5) SELECT h.AvgHighTreatmentMortalityRate, l.AvgLowTreatmentMortalityRate FROM HighTreatmentFacilities h, LowTreatmentFacilities l;"
    },
    {
       "input": "Provide a summary of total opioid fatalities and emergency department visits, along with average population rates for different age groups.",
       "StructurePrompt": "SELECT [aggregate_function1]([column1]) AS [alias1], [aggregate_function2]([column2]) AS [alias2], [aggregate_function3]([column3]) AS [alias3], [aggregate_function4]([column4]) AS [alias4], [aggregate_function5]([column5]) AS [alias5] FROM [table];",
       "ContentPrompt": "SELECT SUM(\"Number of Opioid Fatalities\") AS Total_Opioid_Fatalities, SUM(\"Number of Opioid Overdose Emerg Dpt Cases\") AS Total_Emergency_Department_Visits, AVG(\"Aged <15 Rate\") AS Avg_Under_15_Population_Rate, AVG(\"Aged 15-64 Rate\") AS Avg_15_to_64_Population_Rate, AVG(\"Aged 65+ Rate\") AS Avg_65_and_older_Population_Rate FROM county_data;"
    },
    {
       "input": "Is there a relationship between a county's homicide rates (overall and by race) and its opioid fatality rate?",
       "StructurePrompt": "SELECT corr([column], [comparison_column]) AS [correlation_alias], corr([column2], [comparison_column]) AS [correlation_alias2], corr([column3], [comparison_column]) AS [correlation_alias3], corr([column4], [comparison_column]) AS [correlation_alias4] FROM [table_name];",
       "ContentPrompt": "SELECT corr(\"Homicide Rate (IL)\", \"Opioid Fatality Rate per 100K Capita\") AS Correlation_Overall_Homicide, corr(\"Homicide Rate (Black)\", \"Opioid Fatality Rate per 100K;"
    },
    {
        "input": "Which racial/ethnic group has the highest opioid fatality rate overall?",
        "ContentPrompt": "SELECT MAX(\"White Rate\") AS Max_White_Rate,MAX(\"Black Rate\") AS Max_Black_Rate,MAX(\"Latino Rate\") AS Max_Latino_Rate,MAX(\"Asian Rate\") AS Max_Asian_Rate,MAX(\"Native Hawaiian Rate\") AS Max_Native_Hawaiian_Rate, MAX(\"Indian/Alaskan Native Rate\") AS Max_Indian_Alaskan_Native_Rate FROM county_data;"
    },
    {
        "input": "How does the opioid fatality rate differ between rural and urban counties?",
        "ContentPrompt": "SELECT AVG(\"Opioid Fatality Rate per 100K Capita\") AS AvgOpioidFatalityRate, CASE WHEN \"Rural Rate\" >= 0.5 THEN 'Rural' ELSE 'Urban' END AS CountySetting FROM county_data GROUP BY CountySetting;"
    },
    {
        "input": "What is the relationship between a county's median household income and its opioid fatality rate?",
        "ContentPrompt": "SELECT corr(\"Median Household Income\", \"Opioid Fatality Rate per 100K Capita\") AS CorrelationCoefficient FROM county_data;"
    }
    ]

@st.cache_resource
def get_example_selector():
  example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"]
)
  return example_selector
example_prompt = ChatPromptTemplate.from_messages(
     [
         ("human", "{input}\nSQL query:"),
         ("ai", "{ContentPrompt}")
     ]
 )
top_k = 10
few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     example_selector=get_example_selector(),
     input_variables=["input","top_k", "table_info"]
 )

final_prompt2 = ChatPromptTemplate.from_messages(
    [
         ("system", """You are a PostgreSQL expert. Given an input question, create a syntactically correct PostgreSQL query to run.
         \n Double check the user's query for common mistakes
         - including: using NOT IN with NULL values,
         - using UNION when UNION ALL should have been used,
         - using BETWEEN for exclusive ranges,
         - mismatch in data type predicates,
         - properly quoting identifiers,
         - using the correct number of arguments for functions,
         - casting to the correct data type, and using the proper columns for joins.
        \n Rewrite the query if any mistakes are found; otherwise, reproduce the original query.
        \n Unless otherwise specified, do not return more than {top_k} rows.
        \n REMEMBER all the \"rate\" Columns except \"Opioid Fatality Rate per 100K Capita\" are Population rates NOT fatality rates.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."""),
         few_shot_prompt,
         MessagesPlaceholder(variable_name="messages"),
         ("human", "{input}")
    ]
 )
answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )
@st.cache_data
# Read and process the CSV for column details
def get_column_details():
    try:
        column_description = pd.read_csv("/content/drive/My Drive/County_Data_Columns.csv")
    except FileNotFoundError as e:
        print(f"Failed to read CSV file: {e}")
        return ""

    column_details = ""
    for _, row in column_description.iterrows():
        column_details += f"Column Name: {row['Column']}\nColumn Description: {row['Description']}\n\n"
    return column_details

class Column(BaseModel):
    """Column in SQL database."""
    name: str = Field(description="Name of column in SQL database.")
column_details = get_column_details()
def get_columns(columns: List[Column]) -> List[str]:
    table_columns  = [column.name for column in columns]
    return table_columns

column_details_prompt = (
    "Return the names of ALL the SQL columns that MIGHT be relevant to the user question. "
    "The columns are:\n\n" + column_details +
    "Remember to include ALL POTENTIALLY RELEVANT columns, even if you're not sure that they're needed. "
    "Also, remember, the 'County' column is our primary column which should always be returned with all other relevant columns."
    "Note to also Remember that all the \"rate\" Columns except \"Opioid Fatality Rate per 100K Capita\" are Population rates NOT fatality rates"
)
select_column = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Column, llm, system_message=column_details_prompt) | get_columns
def get_chain():
  print("Creating chain")
  db = SQLDatabase.from_uri(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
  llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
  generate_query2 = create_sql_query_chain(llm, db,final_prompt2)
  execute_query = QuerySQLDataBaseTool(db=db)
  rephrase_answer = answer_prompt | llm | StrOutputParser()
  chain2 = RunnablePassthrough.assign(column_names_to_use=select_column) | (RunnablePassthrough.assign(query=generate_query2).assign(result=itemgetter("query") | execute_query) | rephrase_answer)
  return chain2
def create_history(messages):
    history = ChatMessageHistory()
    for message in messages:
        if message["role"] == "user":
            history.add_user_message(message["content"])
        else:
            history.add_ai_message(message["content"])
    return history
def invoke_chain(question, messages):
    chain = get_chain()
    history = create_history(messages)
    response = chain.invoke({"question": question, "top_k": 3, "messages": history.messages})
    history.add_user_message(question)
    history.add_ai_message(response)
    return response
st.markdown("""<style>body {color: white;background-color: black;}.stTextInput > label, .stButton > button {color: black;} h1 {text-align: center;}</style>
""", unsafe_allow_html=True)

st.title("Chat with Opioid Data")
st.markdown("<h4 style='text-align: right;'>- Pilot project by Sarath Chandra Gentela </h4>", unsafe_allow_html=True)


# Set OpenAI API key from Streamlit secrets
client = OpenAI(api_key="sk-proj-tylU2TViaXKU09BIVf6mT3BlbkFJd1Pfsm4RtuKrOEbVM1d7")

# Set a default model
if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-3.5-turbo-0125"

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
avatar_url_user = "https://edinazephyrus.com/wp-content/uploads/2019/03/Zephyrus-Orange-Pill-Bottle-e1552189222856-900x827.jpg"
avatar_url_robot = "https://img.freepik.com/premium-vector/cute-robot-doctor-white-coat-game-medicine-technologies-future-hospital-treatment-character_124715-101.jpg"
for message in st.session_state.messages:
    with st.chat_message(message["role"],avatar=(avatar_url_robot if message["role"] == "assistant" else avatar_url_user)):
        st.markdown(message["content"])


if prompt := st.chat_input("Ask Anything about Opioid Data"):
        # Process input
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user",avatar= avatar_url_user):
          st.markdown(prompt)
        with st.spinner("Generating response..."):
          with st.chat_message("assistant",avatar = avatar_url_robot):
            response = invoke_chain(prompt,st.session_state.messages)
            st.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})




# Sidebar with help or information section
with st.sidebar:
    st.header("Help Section")
    st.write("""
        **How to use this chatbot:**
        - The dataset used in this chatbot encompasses opioid-related statistics across Illinois's 102 counties, integrating demographic data and several social indicators. It includes metrics such as opioid fatalities, overdose emergency cases, social vulnerability scores, and healthcare access, and more. This data aids in regional analysis and decision-making to combat the opioid crisis.
        - Click **Clear Chat** below to start a new conversation.\n
        - The Chatbot is **prone to Errors**. Since this is made with minimal resources, kindly be patient and refresh the chatbot for a fresh start.\n
        **Example questions:**
        - How does the opioid fatality rate differ between rural and urban counties?
        - What is the relationship between a county's median household income and its opioid fatality rate?
        - Which racial/ethnic group has the highest opioid fatality rate overall?
    """)
    button_style = """<style>.stButton>button {background-color: white;color: black;border: 2px solid #4CAF50;padding: 10px 24px;cursor: pointer;}</style>"""
    st.markdown(button_style, unsafe_allow_html=True)
    if st.button("Clear Chat"):
      st.session_state.messages = []

In [ ]:
from pyngrok import ngrok

# Set the authtoken (Replace 'your_auth_token' with your actual ngrok token)
ngrok.set_auth_token("2g0bQsQV1VNzYglUSlGvjwpnVBC_2KCVsf8HSiXiw7pTZRpME")
# Start a tunnel to the Streamlit port 8501
url = ngrok.connect(8501)
print('Ngrok Tunnel URL:', url)
!streamlit run app.py &